In [1]:
# Settings for notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Show Python version
import platform
platform.python_version()

'3.6.8'

In [2]:
try:
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess

     |████████████████████████████████| 235kB 4.7MB/s 
     |████████████████████████████████| 61kB 23.6MB/s 
     |████████████████████████████████| 3.1MB 45.0MB/s 
     |████████████████████████████████| 2.3MB 19.0MB/s 
     |████████████████████████████████| 61kB 21.1MB/s 
     |████████████████████████████████| 174kB 32.4MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11516 sha256=65804eab6e8e423eefb36476307f5c04b025fcee47adfe118ad1eda6b88a79a6
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
Successfully built PyDispatcher


Setup a pipeline
This class creates a simple pipeline that writes all found items to a JSON file, where each line contains one JSON element.

In [0]:
import json

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('quoteresult.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

Define the spider
The QuotesSpider class defines from which URLs to start crawling and which values to retrieve. I set the logging level of the crawler to warning, otherwise the notebook is overloaded with DEBUG messages about the retrieved data.

In [0]:
import logging

class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
        'http://quotes.toscrape.com/page/2/',
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'quoteresult.json'                        # Used for pipeline 2
    }
    
    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').extract_first(),
                'author': quote.css('span small::text').extract_first(),
                'tags': quote.css('div.tags a.tag::text').extract(),
            }

Start the crawler

In [5]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(QuotesSpider)
process.start()

2019-10-23 05:58:04 [scrapy.utils.log] INFO: Scrapy 1.7.4 started (bot: scrapybot)
2019-10-23 05:58:04 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.7.0, Python 3.6.8 (default, Oct  7 2019, 12:59:55) - [GCC 8.3.0], pyOpenSSL 19.0.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Linux-4.14.137+-x86_64-with-Ubuntu-18.04-bionic
2019-10-23 05:58:04 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'quoteresult.json', 'LOG_LEVEL': 30, 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


Check the files

In [6]:
ll quoteresult.*

-rw-r--r-- 1 root 5551 Oct 23 05:58 quoteresult.jl
-rw-r--r-- 1 root 5573 Oct 23 05:58 quoteresult.json


In [7]:
!tail -n 2 quoteresult.jl

{"text": "\u201cGood friends, good books, and a sleepy conscience: this is the ideal life.\u201d", "author": "Mark Twain", "tags": ["books", "contentment", "friends", "friendship", "life"]}
{"text": "\u201cLife is what happens to us while we are making other plans.\u201d", "author": "Allen Saunders", "tags": ["fate", "life", "misattributed-john-lennon", "planning", "plans"]}




```
# This is formatted as code
```



In [8]:
!tail -n 2 quoteresult.json

{"text": "\u201cLife is what happens to us while we are making other plans.\u201d", "author": "Allen Saunders", "tags": ["fate", "life", "misattributed-john-lennon", "planning", "plans"]}
]

In [9]:
import pandas as pd
dfjson = pd.read_json('quoteresult.json')
dfjson

,author,tags,text
0,Albert Einstein,"[change, deep-thoughts, thinking, world]",“The world as we have created it is a process ...
1,J.K. Rowling,"[abilities, choices]","“It is our choices, Harry, that show what we t..."
2,Albert Einstein,"[inspirational, life, live, miracle, miracles]",“There are only two ways to live your life. On...
3,Jane Austen,"[aliteracy, books, classic, humor]","“The person, be it gentleman or lady, who has ..."
4,Marilyn Monroe,"[be-yourself, inspirational]","“Imperfection is beauty, madness is genius and..."
5,Albert Einstein,"[adulthood, success, value]",“Try not to become a man of success. Rather be...
6,André Gide,"[life, love]",“It is better to be hated for what you are tha...
7,Thomas A. Edison,"[edison, failure, inspirational, paraphrased]","“I have not failed. I've just found 10,000 way..."
8,Eleanor Roosevelt,[misattributed-eleanor-roosevelt],“A woman is like a tea bag; you never know how...
9,Steve Martin,"[humor, obvious, simile]","“A day without sunshine is like, you know, nig..."


In [10]:
!scrapy startproject amazon_reviews_scraping

New Scrapy project 'amazon_reviews_scraping', using template directory '/usr/local/lib/python3.6/dist-packages/scrapy/templates/project', created in:
    /content/amazon_reviews_scraping

You can start your first spider with:
    cd amazon_reviews_scraping
    scrapy genspider example example.com
